# E-commerce data analysis

## Project Description

This project provides an e-commerce example how we can use data to create Business Intelligence which gives us better insights about our customers. 

The idea is that a company approached us, as data analysts, in order to analyse their actual business and to define new actions that lead to a more successful business. To do so, different analysis approches will be used to gain new insights: 
- Exploratory Data Analysis (EDA): a qualitative and quantitative analysis of the actual business that gives us an understanding of the business and the customers. 
- Recency Frequency Monetary (RFM): aims to define the most valuable customers by using three very importan factors
- Customer segmentation: Clustering customers in segments to give us insights about important factors that represent the different customer behavior. For this, Machine Learning K-Means clustering is used. 
- Cohort Analysis: As frequency is an important factor, cohort analysis shows us the monthly churn rates of each market. 
- Survival Analysis: aims to identify the churn rate not over time but over frequency in this case. 
- Recommendation System: aims to increase the retention rate by recommending the right products to each customer. To achieve the recommendation system aims to identify the most similar customers. 
- Chatbot: not part of the analysis but an instrument to implement the analysed best next actions. In this case the chatbot is used to make individualised product offers from the recommendation system. 

## Questions

The analysis of a company with a few 100'000 datasets is a big challenge and also leads to many different questions we need to tackle. Here are the most important ones we want to answer in this project: 
* How can we achieve more profit even though the company is successful?
* Is each market the same or is there a difference of the customer behavior in the different markets?
* What are the most valuable customers?
* How can we cluster the customers?
* Are there factors that explain the different behavior of the customers?
* Which new actions can be defined to improve our business?

## Dataset


For this project I acquired a dataset from Kaggle [Link](https://www.kaggle.com/jr2ngb/superstore-data). But as already said, the approach is applicable to any other product or service providing company. 

The raw dataset contains 51290 rows with 24 columns. The exported CSVs after datacleaning can be found as 'cleaned_csv.csv' in my [github repo](https://github.com/kevforster/Project-Week-8-Final-Project/tree/master/your-project/csv)

## Data Cleaning

The code for the data cleaning can be found in my [github repo](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/Base_cleaned_data.ipynb)

### Import libaries and dataset

For this project a dataset will be used that is from an e-commerce Superstore

In [6]:
import pandas as pd
df=pd.read_csv('../csv/superstore_dataset2011-2015.csv',encoding= 'unicode_escape')

The first insights show us 51290 rows with 24 columns:

<img src="images/image01.png">

We start the data cleaning process by looking at the data types in our dataframe. It shows us many objects and a few float types. 

field         | type
------------- | -------------
Order ID      | object
Order Date    | object
Ship Date     | object
Ship Mode     | object
Customer ID   | object
Customer Name | object
Segment       | object
City          | object
State         | object
Country       | object
Postal Code   | float64
Market        | object
Region        | object
Product ID    | object
Category      | object
Sub-Category  | object
Product Name  | object
Sales         | float64
Quantity      | int64
Discount      | float64
Profit        | float64
Shipping Cost | float64
Order Priority| object

The data seemed to be quite clean and would be good for this analysis project. The dataset only has null values at the Postal Code column. In the next steps we will explain how we handle those missing values. At the end we need a dataframe without any missing value to have the right base for the analysis.

field         | null_values
------------- | -------------
Row ID        |        0
Order ID      |        0
Order Date    |        0
Ship Date     |        0
Ship Mode     |        0
Customer ID   |        0
Customer Name |        0
Segment       |        0
City          |        0
State         |        0
Country       |        0
Postal Code   |    **41296**
Market        |        0
Region        |        0
Product ID    |        0
Category      |        0
Sub-Category  |        0
Product Name  |        0
Sales         |        0
Quantity      |        0
Discount      |        0
Profit        |        0
Shipping Cost |        0
Order Priority|        0

In this case I droped a few columns, as they were not relevant for this project.
- Row ID: too many unique numbers
- Postal Code: Too many unique and missing values. And as we already have the city, we do not need the postal code
- Ship Date: the Order Date will be enough for this project
- State: not relevant for this project

### Data Wrangling each feature

After I got an overview of the data, I started to dig deeper into data wrangling. Each feature was analysed and cleaned.

> Order Date: I change the date to datetime which will make it easier to analyse afterwards

In [12]:
from datetime import datetime as date
df['Order Date'] = pd.to_datetime(df['Order Date'])

> Dropped a few columns for the following reason:
> - Order ID: too many unique values
> - Order Priority: not relevant for this project
> - Region: we already have enough geographical features

In [13]:
df.drop(['Order ID', 'Region', 'Order Priority'],axis=1, inplace=True)

> Renamed the columns to reduce problems with whitespaces. 

In [15]:
df.columns=['Date', 'Ship_Mode','Customer_ID', 'Customer_Name','Segment','City', 'Country', 'Market', 'Product_ID', 'Category', 'Sub_Category', 'Product_Name', 'Sales','Quantity','Discount','Profit','Shipping Cost']

> After analysing Ship Mode I also to decided to drop this column as it is not relevant for me. 

In [18]:
df.drop(['Ship_Mode'],axis=1, inplace=True)

The following boxplot shows an outlier that we found. The average sales amount was below $10'000 whereas the outlier showed an amount above 50'000. An outlier can cause serious problems in statistical analysis. Even though the outlier might not be an error I decided to remove it, as it would give me wrong results if the outlier belongs to a very small market. 

<img src="images/image07.png">

At the end I converted the dataframe to a csv ("cleaned_csv.csv, [Link](http://localhost:8890/tree/modul3/Project-Week-8-Final-Project/your-project/csv)) which will be the base I will work with. Now we have a clean dataset with the most important columns that we will use in this project. 

## Data Analysis

The code for the next data analysis part can be found in my [github repo](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/RFM.ipynb).

The data analysis is divided into different parts where I tried to find the best approach with the best insights:
- Exploratory Data Analysis (EDA)
- Recency Frequency Monetary (RFM)
- Cohort Analysis 
- Survival Analysis


### Exploratory Data Analysis

In statistics, exploratory data analysis (EDA) is an approach to analyse data sets to summarize their main characteristics, often with visual methods.
In a first step we want to see how the business has developed in the last few years. Therefore we need to group the dataset by date-month. The lineplot shows us a positive developing profit curve. 

<img src="images/image4.png">

What are the most selling products? The graph below shows the top 20 products of the store. 

<img src="images/image5.png">

In the Exploratory Data Analysis we could further analyse the transactions by country, growth rates or seasonality. In this case we have a store with a wide range of different products and many different markets that we rather add some more factors that help us to detect the most valuable customers. Therefore we use the recency, frequency, monetary (RFM) concept.

### Recency Frequency Monetary (RFM)

[Code](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/RFM.ipynb)

We aim to detect the customer that make our business so successful at the moment. And at the end we compare the markets to see if we achieve the same success in each market. So, here are the questions:
- Is there any significant difference between the customers of the different markets/countries. 
- Which are the valuable customers and do they still order products from our store?

So those few questions lead me to the RFM concept that is build up by 3 factors:
- Recency: How recently a customer has made a purchase
- Frequency: How often a customer makes a purchase
- Monetary: How much money a customer spends on purchases in total

The RFM value is a marketing analysis tool used to identify a company's or an organization's best customers by using certain measures. 

#### Frequency

The calculation of frequency is the easiest. Each order is one frequency. As we have not grouped the dataframe by customers yet, we just give each order a 1.

In [45]:
df['Frequency']=np.where(df.Quantity>0,1,0)

#### Monetary

The monetary factor is also easy as long as we have it split up by orders. At the moment each sales value is equal to the monetary. Though once we group the dataframe by customers, the monetary factor will represent the total of all sales amounts by each customer. 

In [46]:
df['Monetary']=df.Sales

#### Recency

Recency is the most trickiest one. It represents how recent a customer has ordered something from the store. In case he ordered already a few times, we only take the most recent order date. As datetime is a difficult format we rather convert it into days. 

In [47]:
from datetime import datetime as dt
df['Date']=pd.to_datetime(df['Date'])

What is the last date that was registered of all customers in the whole dataframe? It is the last day in 2014. Therefore we will take the 31st December 2014 as our last day in the timeframe. So we will subtract the most recent order of each customer from that date to measure the recency. 

In [48]:
def recency_days(last_date,row):
    '''input: the date when the order was done, subtracting from the last date registered in the dataframe
    output: difference in days'''
    return abs((last_date - row['Date']).days)

Now we will group the whole dataframe by the unique customers. In this case Frequency and Monetary will be summed up. Whereas Recency only takes the most recent order date and therefore we only display the lowest amount of days from all the orders of the customer. (e.g. Customer XY has made orders 30 days ago, 21 days ago, 7 days ago. In this case we use the "7 days" ago.) 
The table grouped by the Customer looks like the following:

<img src="images/image0101.png">

In a next step we resccale the RFM by using 5 bins for each factor (1-5) and then we add the values up to get the RFM score. 

#### Total RFM score

Now, we will split each factor of the RFM into 5 bins by a scale from 1-5:
- Recency: The more recent a customer ordered from the store, the better. So the lower the number the better. Scale: 5 is the best, 1 is the worst.
- Frequency: The more frequent a customer ordered, the better. Scale: 5 is the best, 1 is the worst.
- Monetary: The higher the monetary amount, the better. Scale: 5 is the best, 1 is the worst.

The following table shows the previous table plus the rescaled RFM, plus the total score in the last column. Obviously the higher this value the better.

<img src="images/image030.png">

Total score values:
- maximum value: 15, most valuable customers
- minimum value: 3, least valuable customers

#### Frequency and Profit by Total RFM score

In the following graph we plotted the Profit and Frequency by the total RFM score. The darker the plot, the higher the RFM score and the more valuable a customer. 
This graph leads us to the assumption that the more frequent customers order the more valuable they are for the company. 

<img src="images/image10.png">

A different graph with profit and total revenue by total RFM score shows us a similar output. The more revenue a customer generates the more profit the company makes at the end. This sounds obvious but the great understanding is that customers that bought recently also generate a lot of revenue and profit at the end (see the color of the plots). This confirms that the company has a good customer base that have ordered recently, buy frequently and generate a lot of revenue. 

<img src="images/image9.png">

#### Clustering with KMeans

In a next step we want to create customer segmentations by using the RFM values. In this unsupervised machine learning approach we use KMeans in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster.
The elbow method shows me the amount of clusters we will use. In this case 3 clusters.

<img src="images/image011.png">

The following graph shows us the same plotting from above just divided into our three defined clusters. From a company perspective it seems like the blue cluster, label 0, is the target for each customer. 

<img src="images/image042.png">

The clustering did not give us new insight. Although we know which are our most valuable customers. In this case we also know that frequency is one of the most critical factor that makes a customer very valuable. For this reason I decided to create a cohort analysis, that shows the retention of a customer divided into months. 

### Cohort Analysis

[Code](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/RFM.ipynb)

Cohort analysis allows a company to “see patterns clearly across the life-cycle of a customer (or user), rather than slicing across all customers blindly without accounting for the natural cycle that a customer undergoes.” By seeing these patterns of time, a company can adapt and tailor its service to those specific cohorts. 

In our cohort we are looking at customers that purchased their first item within a given Cohort_month (y axis). In this case, the cohort shows us a high retention of customers even many months after the first order. Around 40-50% of all customers from 2011-1 cohort month return after 40 months. This is a very positive fact for the company. The graph bellow shows only a section of the whole cohort analysis. 


<img src="images/image6.png">

To see the different characteristics of the different markets, I created a cohort analysis for each market. Although in this paper I only show the cohorts that give us new insights. 
In this case we were able to see that in EMEA the cohort analysis looks very different. This means only 10-15% of EMEA customers from cohort month 2011-1  return after 40 months. This is much lower than the average of all countries as seen above and needs to be investigated.  

<img src="images/image7.png">

After the insights that some markets might have a lower frequency, I decided to use the survival analysis to see which markets and which product segments have a lower order frequency compared to other markets. 

### Survival analysis

[Code](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/Survival_Analysis_Recommendation_system.ipynb)

Survival analysis corresponds to a set of statistical approaches used to investigate the time it takes for an event of interest to occur. In this case we investigate the frequency it takes for a customer stop ordering from our store.

#### Survival graphs by customer

> Attrition rate
>
> Attrition rate or also called churn rate refers to the proportion of contractual customers who leave a supplier during a given time period. 
As we do not have any contracts with customers for our e-commerce store, the first challenge is to define when a customer becomes a "lost customer" and will count as "churned". For this, I calculated the average time between two orders of all customers in days: 80 days. To just use a simple method, I decided to use 180 days without ordering until a customer becomes a "lost customer". 

We learned that the more frequent a customer orders the more profitable he will be for us.
The following graph gives us insights about the customer behavior in terms of frequency in the different markets. 

> 1 = all customers in this market are still my customers

> 0 = all customers in this market are lost customers

<img src="images/image62.png">

So the graph shows us two classes: 
- Canada, Africa, EMEA: The customers in these markets have a lower frequency compared to other markets. 
- APAC, EU, LATAM, US: These markets show a high frequency of orders by their customers. 

In a next step we analyse if there is a difference of frequency in the product segments.

#### Survival graph by Segment

<img src="images/image61.png">

The further filtering gave us a more detailed insight. So the problem in the mentioned countries is found in the segment "Office Supplies". Compared to all the other markets, EMEA, Canada and Africa have a much lower frequency of orders in these section. 

So this leads us to the conclussion that the company should define the right actions to increase the frequency in the markets of Canada, Africa and EMEA. But how can we do this? 

### Next best acions

After the analysis part we found new insights how we could improve our business in some markets. The cohort and survival analysis showed us a lower order frequency in the markets Canada, Africa and EMEA. 

To increase the frequency in these countries the following steps were defined:
- Find similar customers: A machine learning approach calculates the most similar customers by using the history data of product orders and the quantity
- Individualised product recommendation: Creating a recommendation system based on the customer similarity. 
- Chatbot, offer the products: Integrate a chatbot to achieve a personalised product offering for each customer. 

This is an approach to help a customer to find the right products in our store and therefore also order products more frequently. 

<img src="images/image040.png">

## Recommendation System

[Code](https://github.com/kevforster/Project-Week-8-Final-Project/blob/master/your-project/notebooks/Survival_Analysis_Recommendation_system.ipynb)


We put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers.

For the sake of simplicity we will just focus on Africa for the next steps. But this concept would also work for the other markets.

### Similar customers

In a first step we use a machine learning approach to find similar customers that have bought similar products in a similar amount of quantity. 

>What kind of distance metric should we use?
>
>- Euclidean distance is the most common use of distance. In most cases when people said about distance, they will refer to Euclidean distance. Euclidean distance is also known as simply distance. When data is dense or continuous, this is the best proximity measure.
The Euclidean distance between two points is the length of the path connecting them.The Pythagorean theorem gives this distance between two points.
>- Manhattan distance is a metric in which the distance between two points is the sum of the absolute differences of their Cartesian coordinates. In a simple way of saying it is the total suzm of the difference between the x-coordinates  and y-coordinates.
>- The Minkowski distance is a generalized metric form of Euclidean distance and Manhattan distance.
>- Cosine similarity metric finds the normalized dot product of the two attributes. By determining the cosine similarity, we would effectively try to find the cosine of the angle between the two objects. The cosine of 0° is 1, and it is less than 1 for any other angle.

We are using the Euclidean distance as we are only looking for the shortest distance between two points. So, the following matrix shows us the distance between two customers. The closer the number to 1 the similar  the customers, whereas 0 is the lowest similarity score. 

<img src="images/image013.png">

The matrix in a heatmap shows a more colourful result. In this case the lighter the cell the similar the two customers. (The graph below represents only a section of a huge heatmap)

<img src="images/image070.png">

Once we had our similarity matrix, then we could produce recommendations for each user and pack all the recommendations into a data frame. Here is an example of recommended products for one customer:

<img src="images/image071.png">

We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer has already purchased any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased yet. These will be the items that we recommend to the customer.

### Recommendation

So in this case we created a list of the top 5 recommendations for each customer that he has not bought before. 

<img src="images/image19.png">

After knowing what products the customer would like, we only have to somehow be able to recommend these products to him. A chatbot will help us to do so. 

## Chatbot

### Definition

A chatbot aims to be an intelligent virtual assistant that fully automates conversations at every stage of a marketing and sales funnel. 
Here is an example of a chatbot:


<img src="images/image015.png">

### The result

In our project case we wanted to use the chatbot to be able to recommend the products to the customer, only by knowing the customer's name. 
This way we can help the user to find the right products in this huge store in a very efficient way. 
My POC only focused on the functionalty and not on any frontend. So apologise for the GUI. 

As you can see the user starts with a normal discussion by saying 'Hello', 'Hi' or anything similar. The chatbot understands this as  'greetings' and gives the right answer. In the second part, the user asks for help where the chatbot gives a standard answer again. But once the chatbot knows who the user is, it gets interesting. So, by knowing the user, the chatbot can give personalised answers by recommending the products. I built it the way that the chatbot recommends randomly one of the top 5 recommendations of the customer. We could further optimise it by adding also the link to the product so we will lead the user directly to the recommended product.  

<img src="images/image016.png">

Obviously the chatbot usually does not ask for your name, but as we all know there are other ways to identify your user (e.g. IP address, logged in state etc.). Also by knowing the IP location we can already produce personalised content in the chatbot. 
So this POC showed me the huge potential of a chatbot for the future. At the moment we just face the problem that chatbots are often not smart enough or are not integrated in the sales funnel good enough because of static processes in the backend. Therefore we need to change core systems first to make a a chatbot smart enough to be able to help customers. 

## Conclusions

Started by analysing the data of a very successful big store, we were able to find markets with potential to increase its profit. 
* The RFM concept showed us a nice customer base that achieves a high RFM score. This means our valuable customers have a high order frequency, high monetary and a low recency in days. These are critical factors in terms of a successful business. 
* The cohort analysis showed an overall high retention by monthly cohorts. Around 40-50% of all customers from 2011-1 cohort month return even after 40 months. Although the analysis also shwoed a different customer behavior in the different markets. In the EMEA, Canada and Africa markets we only achieve 10-15% retention during the same timeframe
* The survival analysis confirmed the lower order frequency in the  markets Canada, EMEA and Africa. In detail, the problem of the lower amount has been found in the product segment 'Office Supplies'.  
* To increase the frequency in those markets we created a recommendation system that defines personalised product recommendations based on customer similarity. 
* A chatbot will be integrated in our sales funnel that should lead to an increase in order frequency. As an 'intelligent virtual assistent', the chatbot will make personalised product recommendations and will therfore optimise the users' ordering process. 
* Outlook: Unfortunately there are not many smart chatbots implemented yet because of static processes in the backend. But once the changes in the core systems are made, the integration of a chatbot can not only generate more revenue but can also lead to a great customer experience.



## Repo structure

There are 4 folders:
1. images: contains all the screenshots and images that i used for the final paper
2. CSV: contains all the csv that i used during my analysis. After each step (cohort analysis, recommendation system etc.) i created a new csv.
3. notebooks, I split the notebooks up: 
    - Data cleaning (Base_cleaned_data.ipynb), 
    - Data analysis with RFM KMeans clustering and cohort analysis (RFM.ipynb), 
    - Survival analysis and Recommendation System (Survival_Analysis_Recommendation_system.ipynb)
    - Final paper with all the insights and graphs consolidated. This paper does not contain all the codes but all the insights and graphs. (Final_Paper.ipynb)
4. Chatbot: for this last chapter of the project I created an own folder. In there you will find the JSON for the chatbot and all necessary codes in the file chatbot.ipynb

## Links

[Short presentation of the project](https://docs.google.com/presentation/d/1Hl1xUA83V_0pgmEQGNUbdxnrXd7iUZsLCne7AKhhBEA/edit?usp=sharing) 

[Repository](https://github.com/kevforster/Project-Week-8-Final-Project/tree/master/your-project)